In [59]:
import numpy as np
import sympy as sp
from scipy import optimize as opt
from sympy.utilities.lambdify import lambdify
from sympy import Matrix
from tkinter import *
import time

edge = np.genfromtxt('csv/adjacency.csv', delimiter=",").astype(np.int64)
edge_num = len(edge)
P = np.genfromtxt('csv/mdSpace.csv', delimiter=",")
node_num, high_dim = P.shape

dim = 2

def genE():
    L = np.sqrt(np.genfromtxt('csv/eigVals.csv', delimiter=",")[0:high_dim])
    base = np.zeros(high_dim * dim).reshape(dim, high_dim)
    for i in range(high_dim): base[i % dim][i] = 1
    E = base*L
    return E

E = genE()

Xs = np.zeros(node_num)
Ys = np.zeros(node_num)
boundingX = 0
boundingY = 0

def update_points():
	for i in range(node_num):
		global Xs, Ys, boundingX, boundingY
		p0 = P[i, 0:high_dim]
		Xs[i] = np.dot(p0, E[0]) ; Ys[i] = np.dot(p0, E[1])
	boundingX = max([np.amax(Xs), np.amin(Xs)]) * 2
	boundingY = max([np.amax(Ys), np.amin(Ys)]) * 2

update_points()

print("init: ready")

init: ready


In [61]:
E

array([[  4.12843841e+00,   0.00000000e+00,   7.07106781e-01,
          0.00000000e+00,   7.07106781e-01,   0.00000000e+00,
          7.07106781e-01,   0.00000000e+00,   7.07106781e-01,
          0.00000000e+00,   7.07106781e-01,   0.00000000e+00],
       [  0.00000000e+00,   4.12843841e+00,   0.00000000e+00,
          7.07106781e-01,   0.00000000e+00,   7.07106781e-01,
          0.00000000e+00,   7.07106781e-01,   0.00000000e+00,
          7.07106781e-01,   0.00000000e+00,   2.10734243e-08]])

In [60]:
print(Xs[14])
print(Ys[14])

-6.03749214874
3.25129988464


In [17]:
# sympy
a1,b1,c1,a2,b2,c2,t,s = sp.symbols('a1 b1 c1 a2 b2 c2 t s')   # variables
x2_s,y2_s = sp.symbols('x2_s y2_s')  # values
P_i = sp.MatrixSymbol('P_i', high_dim, 1)
E1 = sp.MatrixSymbol('E1', high_dim, 1)
E2 = sp.MatrixSymbol('E2', high_dim, 1)
var = (x2_s,y2_s,P_i,E1,E2,a1,b1,c1,a2,b2,c2,t,s)
inputs = [x2_s,y2_s,P_i,E1,E2,a1,b1,c1,a2,b2,c2,t,s]
_E1 = sp.Matrix(a1*E1 + b1*E2 + c1*P_i)
_E2 = sp.Matrix(a2*E1 + b2*E2 + c2*P_i)
R = sp.Matrix(s*E1 + t*E2)

f = Matrix([
		_E1.dot(_E1) - 1,
		_E2.dot(_E2) - 1,
		_E1.dot(_E2),
		R.dot(R) - 1,
		_E1.dot(R) - sp.Matrix(E1).dot(R),
		_E2.dot(R) - sp.Matrix(E2).dot(R),
		sp.Matrix(P_i).dot(_E1) - x2_s,
		sp.Matrix(P_i).dot(_E2) - y2_s
		])

func = sp.Matrix.norm(f)
lam_f = lambdify(var, func, 'numpy')

def lam(x2, y2, p, e_1, e_2):
    return lambda a1,b1,c1,a2,b2,c2,t,s: \
        lam_f(x2, y2, p, e_1, e_2, a1, b1, c1, a2, b2, c2, t, s)

arr_init = np.array([1, 0, 0, 0, 1, 0, 1, 1])
print("lambda: ready")


lambda: ready


In [51]:
[1 if type(v)==Symbol else 0 for v in inputs]

[1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

In [65]:
from sympy.printing.theanocode import theano_function as th_func
#dicts = [(v,'float32') if type(v)==Symbol else (v,'numpy.ndarray') for v in inputs]
#dtypes = {v:t for (v,t) in dicts}
dtypes = {v: 'float64' for v in inputs}
theo_f = th_func(inputs,[func],dtypes=dtypes)

KeyError: <class 'sympy.matrices.expressions.matexpr.MatrixElement'>

In [64]:
type(E[0].reshape(high_dim,1))

numpy.ndarray

In [55]:
######## Graph Drawing ########
identifier = "14"
arr = np.array([1, 1, 1, 1, 1, 1, 1, 1])

f2 = lam(0, 0, P[int(identifier)].reshape(d,1), e1, e2)
def g(args): return f2(*args)
res = opt.minimize(g, arr, method='L-BFGS-B')
print(res)
e1 = res.x[0] * temp1 + res.x[1] * temp2 + res.x[2] * P[int(identifier)].reshape(d,1)
e2 = res.x[3] * temp1 + res.x[4] * temp2 + res.x[5] * P[int(identifier)].reshape(d,1)
temp1 = e1
temp2 = e2
update_points()
print(Xs[14])
print(Ys[14])

NameError: name 'd' is not defined

In [18]:
aray = np.array([1,2,3,4,5,6,7,8,9,10,11,12])
aray[2::3] = 0
aray[1::3] = 0
aray

array([ 1,  0,  0,  4,  0,  0,  7,  0,  0, 10,  0,  0])

In [19]:
import numpy as np
import sympy as sp
from scipy import optimize as opt
from matplotlib import pyplot as plt
from sympy.utilities.lambdify import lambdify
from sympy import Matrix

#  initialize
edge = np.genfromtxt('csv/adjacency.csv', delimiter=",").astype(np.int64)
P = np.genfromtxt('csv/mdSpace.csv', delimiter=",")
n = len(P)
L = np.genfromtxt('csv/eigVals.csv', delimiter=",")
L_pos = np.array([L[i] if L[i] > 0 else 0 for i in range(n)])
d = np.count_nonzero(L_pos)
Ln = np.sqrt(L_pos)

f1 = np.array(Ln[0:d])
f2 = np.array(Ln[0:d])
f3 = np.array(Ln[0:d])
f1[0::3] = 0 ; f1[1::3] = 0
f2[1::3] = 0 ; f2[2::3] = 0
f3[2::3] = 0 ; f3[0::3] = 0
e1 = (f1 / np.linalg.norm(f1)).reshape(d,1)
e2 = (f2 / np.linalg.norm(f2)).reshape(d,1)
e3 = (f3 / np.linalg.norm(f3)).reshape(d,1)
temp1 = e1
temp2 = e2
temp3 = e3

Xs = np.zeros(n)
Ys = np.zeros(n)
Zs = np.zeros(n)

def update_points():
    for i in np.arange(n):
        global Xs, Ys, Zs
        p0 = P[i, 0:d]
        Xs[i] = np.dot(p0, e1)
        Ys[i] = np.dot(p0, e2)
        Zs[i] = np.dot(p0, e3)

update_points()

print("init: ready")
print(Xs[14],Ys[14],Zs[14])

init: ready
-0.135780740523 -1.37563426606 0.767244058045


In [22]:
identifier = "14"

# sympy
a1,b1,c1,d1,a2,b2,c2,d2,a3,b3,c3,d3,t,s,u = sp.symbols('a1 b1 c1 d1 a2 b2 c2 d2 a3 b3 c3 d3 t s u')   # variables
x2_s,y2_s,z2_s = sp.symbols('x2_s y2_s z2_s')  # values
P_i = sp.MatrixSymbol('P_i', d, 1)
E1 = sp.MatrixSymbol('E1', d, 1)
E2 = sp.MatrixSymbol('E2', d, 1)
E3 = sp.MatrixSymbol('E3', d, 1)
_var = (x2_s,y2_s,z2_s,P_i,E1,E2,E3,a1,b1,c1,d1,a2,b2,c2,d2,a3,b3,c3,d3,t,s,u)

_E1 = sp.Matrix(a1*E1 + b1*E2 + c1*E3 + d1*P_i)
_E2 = sp.Matrix(a2*E1 + b2*E2 + c2*E3 + d2*P_i)
_E3 = sp.Matrix(a3*E1 + b3*E2 + c3*E3 + d3*P_i)
R = sp.Matrix(t*E1 + s*E2 + u*E3)

_f = Matrix([
		_E1.dot(_E1) - 1,
		_E2.dot(_E2) - 1,
		_E3.dot(_E3) - 1,
		_E1.dot(_E2),
		_E2.dot(_E3),
		_E3.dot(_E1),
		R.dot(R) - 1,
		_E1.dot(R) - sp.Matrix(E1).dot(R),
		_E2.dot(R) - sp.Matrix(E2).dot(R),
		_E3.dot(R) - sp.Matrix(E3).dot(R),
		sp.Matrix(P_i).dot(_E1) - x2_s,
		sp.Matrix(P_i).dot(_E2) - y2_s,
		sp.Matrix(P_i).dot(_E3) - z2_s
		])

_func = sp.Matrix.norm(_f)

_lam_f = lambdify(_var, _func, 'numpy')

def _lam(x2,y2,z2,p,e_1,e_2,e_3):
    return lambda a1,b1,c1,d1,a2,b2,c2,d2,a3,b3,c3,d3,t,s,u: \
        _lam_f(x2,y2,z2,p,e_1,e_2,e_3,a1,b1,c1,d1,a2,b2,c2,d2,a3,b3,c3,d3,t,s,u)

    
ons = np.ones(15)
_arr = np.array(ons)

_f2 = _lam(0, 0, 0, P[int(identifier)].reshape(d,1), e1, e2, e3)
def _g(args): return _f2(*args)
res = opt.minimize(_g, _arr, method='L-BFGS-B')
print(res)
e1 = res.x[0] * temp1 + res.x[1] * temp2 + res.x[2] * temp3 + res.x[3] * P[int(identifier)].reshape(d,1)
e2 = res.x[4] * temp1 + res.x[5] * temp2 + res.x[6] * temp3 + res.x[7] * P[int(identifier)].reshape(d,1)
e3 = res.x[8] * temp1 + res.x[9] * temp2 + res.x[10] * temp3 + res.x[11] * P[int(identifier)].reshape(d,1)
update_points()
print(Xs[14],Ys[14],Zs[14])

      fun: 6.9348480821558674e-06
 hess_inv: <15x15 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.47280315,  0.1479708 , -0.46911059, -0.03828469,  0.14711884,
        0.19723734,  0.09504176, -0.4375587 ,  0.436957  ,  0.31229612,
       -0.13059697, -0.14977589, -0.00439372, -0.16080881,  0.20957697])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 2336
      nit: 104
   status: 0
  success: True
        x: array([ 0.96034818,  0.75434511, -0.5929625 ,  0.51933612,  0.23695358,
       -0.02537001,  1.07883479, -0.26572628, -0.22593551,  1.85861003,
       -0.65209731,  0.96837802,  0.77793402, -0.3605194 , -0.51463019])


-1.00678736542e-07 -9.81173794042e-07 -3.4502068666e-07
